In [3]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [36]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path
import datacompy
from datetime import datetime, date
from parse import parse
from glob import glob 


In [37]:
pd.set_option('display.max_rows', 50)  # Disable row limit
pd.set_option('display.max_columns', 40)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
# Set the display precision to 2 decimal places  
pd.set_option('display.precision', 2)  
  
# Or set the float format to always show 2 decimal places  
pd.set_option('display.float_format', '{:.2f}'.format)  


In [38]:
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"


In [96]:
DT_LEN = len("2024-12-12 18:34:25")
merged = pd.DataFrame()
for file_name in sorted(glob("output/*.jsonl")):
    print(f"Reading {file_name}")
    result = parse("output/{date} {time} {postfix}", file_name)
    if result is None:
        print(f"Failed to parse {file_name}")
        continue
    if result["postfix"].startswith("fast"):
        print(f"Skipping fast run")
        continue
    dt = date.fromisoformat(result["date"])
    
    newdf = pd.read_json(file_name, lines=True)
    assert newdf.ad_id.duplicated().sum() == 0, "Expected no duplicates"
    newdf.set_index("ad_id", inplace=True)
    if merged.empty:
        merged = newdf
        merged["delete_date"] = np.nan
        continue
    
    condition = ~merged.index.isin(newdf.index) & merged['delete_date'].isna()
    merged.loc[condition, 'delete_date'] = dt
    new_deleted_count = condition.sum()

    new = newdf.index.difference(merged.index)
    merged = pd.concat([merged, newdf.loc[new]])
    print(f"Total: {len(merged)} read: {len(newdf)} new: {len(new)} deleted: {new_deleted_count}")
df = merged

Reading output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.jsonl


Reading output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 35559 read: 34669 new: 745 deleted: 890
Reading output/2024-12-15 18:00:14 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 35734 read: 34398 new: 175 deleted: 490
Reading output/2024-12-16 23:13:52 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 36281 read: 34510 new: 547 deleted: 647
Reading output/2024-12-17 21:31:02 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 37085 read: 34681 new: 804 deleted: 800
Reading output/2024-12-18 23:01:10 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 37577 read: 34809 new: 492 deleted: 464
Reading output/2024-12-19 22:09:26 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 38047 read: 33723 new: 470 deleted: 1661
Reading output/2024-12-22 21:49:27 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 39161 read: 33255 new: 1114 deleted: 1833
Reading output/2024-12-23 19:08:45 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 39512

In [101]:
def add_city_disctrict_cols(df):
    df[["city", "district"]] = df.location.str.split(",", expand=True)
    return df
df = add_city_disctrict_cols(df)
df.iloc[0].T

url                                                 https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
title                                                                                     4-bedroom detached house to rent
price                                                                                                              1650.00
original_price                                                                                                         NaN
price_per_sqm                                                                                                          NaN
location                                                                                               Larnaca, Dromolaxia
posted                                                                                                     Yesterday 20:57
reference_number                                                                                                       NaN
views           

In [102]:
# remove invalid properties
df = utils.filter_in(df, "`Property area` > 10 and `Property area` < 300")

removing 11414/43667 rows


In [103]:
df.price_per_sqm = df.price / (df["Property area"]).round(2)

/tmp/ipykernel_3698405/3899145994.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [105]:
# selected_df = saledf[saledf.price < 5000]
# print(f"removed {len(saledf) - len(selected_df)} rows")
# df = selected_df

In [106]:
# selected_df = df.query("`Bedrooms` in ('3', '2', '1', 'Studio')")
# print(f"removed {len(df) - len(selected_df)} rows")
# df = selected_df

In [115]:
def stats(saledf, rentdf):
    salestatdf = saledf.query("Condition == 'Resale'").groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    rentstatdf = rentdf.groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    common_index = salestatdf.index.intersection(rentstatdf.index)
    apr = rentstatdf.loc[common_index] / salestatdf.loc[common_index] * 12 * 100
    return pd.concat([apr, rentstatdf, salestatdf], axis=1, keys=['apr', 'rent', 'sale']).sort_values(by=('apr', '50%'), ascending=False).dropna()
    
print("apparts")
display(stats(df.query("cat1 == 'Apartments, flats for sale'"), df.query("cat1 == 'Apartments, flats to rent'")))
print("houses")
display(stats(df.query("cat1 == 'Houses for sale'"), df.query("cat1 == 'Houses to rent'")))

apparts


apr  \
                        count   
city      Bedrooms              
Limassol  6 and more  1200.00   
          5           2160.00   
Larnaca   Studio      1623.53   
          4            694.74   
Paphos    Studio      4440.00   
Larnaca   3            828.45   
Limassol  Studio     15428.57   
Larnaca   2           1797.17   
          1           1959.63   
Limassol  1           5424.00   
Famagusta 3            254.55   
Paphos    1           1889.66   
Nicosia   1           7451.85   
          Studio      6000.00   
Paphos    2           2038.19   
Nicosia   3           2058.20   
          2           3132.20   
Limassol  2           2758.58   
Famagusta 1            564.71   
Paphos    3           1667.53   
Nicosia   4           2325.00   
Famagusta 2            564.10   
Limassol  3           2115.03   
Famagusta Studio      2400.00   
Limassol  4           1346.67   
Paphos    4           2400.00   

                            \
                      mean   
city      Bedrooms           
Limassol  6 and more 32.78   
          5           6.54   
Larnaca   Studio     10.75   
          4           8.52   
Paphos    Studio      8.77   
Larnaca   3           6.44   
Limassol  Studio      7.85   
Larnaca   2           8.17   
          1           6.51   
Limassol  1           5.24   
Famagusta 3           4.77   
Paphos    1           6.02   
Nicosia   1           6.13   
          Studio      7.14   
Paphos    2           5.84   
Nicosia   3           5.73   
          2           5.86   
Limassol  2           5.54   
Famagusta 1           4.78   
Paphos    3          37.03   
Nicosia   4           4.39   
Famagusta 2           4.13   
Limassol  3           4.90   
Famagusta Studio      1.63   
Limassol  4           3.77   
Paphos    4           3.10   

                             \
                        std   
city      Bedrooms            
Limassol  6 and more 185.96   
          5            3.12   
Larnaca   Studio      10.57   
          4            8.80   
Paphos    Studio      10.96   
Larnaca   3            4.11   
Limassol  Studio      10.54   
Larnaca   2           86.51   
          1            5.20   
Limassol  1            3.71   
Famagusta 3            4.03   
Paphos    1            5.33   
Nicosia   1            6.50   
          Studio       7.87   
Paphos    2            5.68   
Nicosia   3            5.07   
          2            5.50   
Limassol  2           29.12   
Famagusta 1            4.88   
Paphos    3          372.23   
Nicosia   4            2.45   
Famagusta 2            1.70   
Limassol  3            4.73   
Famagusta Studio       0.20   
Limassol  4            3.72   
Paphos    4            1.54   

                             \
                        min   
city      Bedrooms            
Limassol  6 and more   2.13   
          5            8.14   
Larnaca   Studio       9.61   
          4            8.89   
Paphos    Studio       8.89   
Larnaca   3            7.43   
Limassol  Studio       3.61   
Larnaca   2            4.94   
          1            6.80   
Limassol  1            1.21   
Famagusta 3            5.36   
Paphos    1            6.89   
Nicosia   1          237.50   
          Studio       5.08   
Paphos    2            4.51   
Nicosia   3            6.47   
          2            5.97   
Limassol  2            1.67   
Famagusta 1            8.26   
Paphos    3            0.50   
Nicosia   4            2.49   
Famagusta 2            6.38   
Limassol  3            4.87   
Famagusta Studio      10.13   
Limassol  4            3.78   
Paphos    4            5.76   

                            \
                       25%   
city      Bedrooms           
Limassol  6 and more 17.83   
          5           6.35   
Larnaca   Studio     10.06   
          4           9.66   
Paphos    Studio      8.48   
Larnaca   3           6.28   
Limassol  Studio      6.66   
Larnaca   2           7.02   
          1           6.51   
Limassol  1           5.77   
Famagus

houses


apr  \
                     count   
city      Bedrooms           
Limassol  1        7440.00   
Nicosia   5         276.92   
Larnaca   5         292.68   
Nicosia   1        2100.00   
Paphos    1        2800.00   
Larnaca   4         463.92   
          3         822.41   
Paphos    5         553.85   
Larnaca   2        1098.80   
Limassol  2        1578.38   
          5         861.18   
Nicosia   2        1353.85   
          3         914.59   
Larnaca   1         750.00   
Limassol  3        1625.00   
          4         924.04   
Paphos    6        1200.00   
          4         939.62   
Nicosia   4         572.09   
Paphos    3         796.19   
          2        1167.86   
Limassol  6         600.00   
Famagusta 5          92.31   
          3         474.07   
          2         523.64   
          4         316.98   

                         \
                   mean   
city      Bedrooms        
Limassol  1        9.19   
Nicosia   5        6.34   
Larnaca   5        5.89   
Nicosia   1        5.99   
Paphos    1        4.63   
Larnaca   4        5.25   
          3        6.38   
Paphos    5        5.67   
Larnaca   2        5.95   
Limassol  2        5.53   
          5        5.71   
Nicosia   2        5.48   
          3        5.48   
Larnaca   1        6.49   
Limassol  3        5.23   
          4        4.68   
Paphos    6        4.77   
          4        3.91   
Nicosia   4        4.43   
Paphos    3        4.81   
          2        4.95   
Limassol  6        7.32   
Famagusta 5        3.92   
          3        4.05   
          2        3.92   
          4        3.60   

                          \
                     std   
city      Bedrooms         
Limassol  1         5.85   
Nicosia   5         3.55   
Larnaca   5         3.66   
Nicosia   1         7.60   
Paphos    1         1.72   
Larnaca   4         3.02   
          3         7.67   
Paphos    5         5.76   
Larnaca   2         2.58   
Limassol  2         4.55   
          5         5.72   
Nicosia   2         3.57   
          3         5.57   
Larnaca   1         6.66   
Limassol  3         7.54   
          4         3.63   
Paphos    6         2.09   
          4         1.21   
Nicosia   4         1.42   
Paphos    3         5.45   
          2         4.29   
Limassol  6        15.62   
Famagusta 5         1.02   
          3         2.73   
          2         2.11   
          4         2.96   

                          \
                     min   
city      Bedrooms         
Limassol  1         8.82   
Nicosia   5        11.75   
Larnaca   5        10.34   
Nicosia   1         4.18   
Paphos    1         7.73   
Larnaca   4         6.49   
          3         9.09   
Paphos    5        14.40   
Larnaca   2        10.71   
Limassol  2         5.36   
          5        12.71   
Nicosia   2        15.67   
          3         8.61   
Larnaca   1         9.48   
Limassol  3         5.91   
          4         8.27   
Paphos    6         9.83   
          4         4.61   
Nicosia   4         4.86   
Paphos    3         7.30   
          2         6.00   
Limassol  6        10.03   
Famagusta 5        11.08   
          3         7.35   
          2         4.25   
          4         6.84   

                          \
                     25%   
city      Bedrooms         
Limassol  1        12.62   
Nicosia   5         6.79   
Larnaca   5         6.51   
Nicosia   1         5.36   
Paphos    1         6.46   
Larnaca   4         6.03   
          3         5.89   
Paphos    5         5.16   
Larnaca   2         7.41   
Limassol  2         5.44   
          5         7.00   
Nicosia   2         6.35   
          3         5.40   
Larnaca   1         6.86   
Limassol  3         5.56   
          4         5.62   
Paphos    6         6.54   
          4         4.38   
Nicosia   4         5.10   
Paphos    3         4.55   
          2         5.13   
Limassol  6         5.28   
Famagusta 5         6.15   
          3   

In [129]:
rentdf = df.query("cat1 == 'Apartments, flats for sale' and price < 1000000")
px.histogram(rentdf, x="price", barmode='overlay', facet_col="city", color="Bedrooms", title="Apts for sale per city/bedrooms").show()


In [130]:
rentdf = df.query("cat1 == 'Apartments, flats to rent' and price < 3000")
px.histogram(rentdf, x="price", barmode='overlay', facet_col="city", color="Bedrooms", title="Apts for rent per city/bedrooms").show()


In [111]:
df.sort_values("views", ascending=False)


,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,...,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,delete_date,city,district
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4025924,https://www.bazaraki.com/adv/4025924_1-bedroom-apartment-to-rent/,Studio apartment to rent,600.00,NaN,17.14,"Limassol, Trimiklini",18.11.2024 13:34,NaN,27310,34.85,32.91,False,Cyprus real estate to rent,"Apartments, flats to rent",35.00,Not allowed,Apartment,Covered,NaN,Fully Furnished,...,4730.00,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Limassol,Trimiklini
1241192,https://www.bazaraki.com/adv/1241192_serviced-offices-for-rent/,Serviced offices in limassol,650.00,NaN,59.09,"Limassol, Limassol - Agia Zoni",05.12.2024 15:10,NaN,22598,34.69,33.04,False,Cyprus real estate to rent,Commercial property,11.00,NaN,Offices,NaN,NaN,NaN,...,3027.00,NaN,NaN,NaN,€59 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Limassol,Limassol - Agia Zoni
4095853,https://www.bazaraki.com/adv/4095853_1-bedroom-detached-house-for-sale/,2-bedroom detached house fоr sаle,70000.00,NaN,1272.73,"Paphos, Filousa Kelokedaron",29.11.2024 18:41,NaN,18964,NaN,NaN,False,Cyprus real estate for sale,Houses for sale,55.00,NaN,Detached house,NaN,156.00,Fully Furnished,...,NaN,NaN,NaN,Resale,€1.273 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02,Paphos,Filousa Kelokedaron
2291746,https://www.bazaraki.com/adv/2291746_2-preserved-houses-2-diateretees-oikies/,3-bedroom detached house fоr sаle,75000.00,80000.00,294.12,"Nicosia, Moni Kato",12.11.2024 07:25,NaN,18630,35.07,33.10,False,Cyprus real estate for sale,Houses for sale,255.00,NaN,Detached house,No,NaN,Unfurnished,...,2776.00,NaN,NaN,Resale,€294 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-14,Nicosia,Moni Kato
4256623,https://www.bazaraki.com/adv/4256623_2-bedroom-detached-house-to-rent/,2-bedroom detached house to rent,700.00,NaN,5.83,"Limassol, Kaminaria",02.12.2024 06:19,NaN,16077,NaN,NaN,False,Cyprus real estate to rent,Houses to rent,120.00,Not allowed,Detached house,No,NaN,Fully Furnished,...,NaN,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02,Limassol,Kaminaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599290,https://www.bazaraki.com/adv/5599290_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,460000.00,NaN,4466.02,"Limassol, Germasogeia",14:16,NaN,0,34.71,33.08,False,Cyprus real estate for sale,"Apartments, flats for sale",103.00,NaN,Apartment,Covered,NaN,NaN,...,NaN,NaN,551733,Under construction,€4.466 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Limassol,Germasogeia
5599297,https://www.bazaraki.com/adv/5599297_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,385000.00,NaN,3318.97,"Limassol, Ypsonas",14:16,NaN,0,34.68,32.97,False,Cyprus real estate for sale,"Apartments, flats for sale",116.00,NaN,Apartment,Covered,NaN,NaN,...,NaN,NaN,639614,Under construction,€3.319 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Limassol,Ypsonas
5599299,https://www.bazaraki.com/adv/5599299_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,1189500.00,NaN,11548.54,"Limassol, Germasogeia",14:16,NaN,0,34.70,33.10,False,Cyprus real estate for sale,"Apartments, flats for sale",103.00,NaN,Apartment,Covered,NaN,NaN,...,NaN,NaN,693389,Under construction,€11.549 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Limassol,Germasogeia


In [19]:
saledf = pd.read_json("output/_real-estate-for-sale_apartments-flats__20241207_133456.jsonl", lines=True)
saledf

ValueError: Expected object or value

In [20]:
saledf.iloc[0].T

url                   https://www.bazaraki.com/adv/5311477_6-bedroom-detached-house-for-sale/
title                                                       6-bedroom detached house fоr sаle
price                                                                                 5900000
original_price                                                                            NaN
price_per_sqm                                                                         7338.31
                                                       ...                                   
Registration block                                                                        NaN
Area                                                                                      NaN
Pick a point                                                                              NaN
city                                                                                 Limassol
district                                                    

In [21]:
saledf[["city", "district"]] = saledf.location.str.split(",", expand=True)

/tmp/ipykernel_3698405/1255282723.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

